# Linear model of stock price changes during COP

In [1]:
from datetime import timedelta, datetime
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from pathlib import Path
import seaborn as sns
from sklearn.model_selection import train_test_split

In [2]:
from scipy.stats import ttest_1samp, gstd, kstest, ttest_ind
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm

In [3]:
version = "17"
# Use this option to test how good the model is, set to false to get the best results. 
do_Test_train_split = False
# Use to get a company-year linear trend, set to false for a company trend and a yearly trend across all companies.
year_company_data = True
# This is a switch to allow investigation of multiple sets of companies. Valid options include:
# "renewable_all", "fossil_34", "control_34" (top 20 of any kind), "constsust_34" 
# (top 34 with neutral ethics ratings), "greencred_34", "fossil_100", 
# "all_companies" (top 100 with any ethics ratings, uses rating as a continuous variable)
# and a few assorted variables for one-off investigations.
filetype = "constsust_34"
# Only include companies that have data before this date:
require_time_start = pd.to_datetime('2011-01-01')
# Do we want to include a period some days before/after the dictionary of events?
# You will need padding for 1-day events (e.g. OPEC meetings, IPCC reports, OilSpills)
padafter = 0
padbefore = 0
# If this variable is not empty/False, we switch to studing a different time series. Options include None = COPs, 
# "OilSpill", "OPEC", "OPEC_Conference"
# "OPEC_28" (only 28 equally spaced OPEC meetings so stats are easier), "IPCC"
copOrOther = "OPEC"
# If this is a string (probably "constsust_34"), we subtract the average fractional change in this filetype 
# before calculating statistics.
norm_group = "constsust_34"
# Do we want to remove data from days with stock splits and low volume? (remove days with vol<1000 and the day before and after)
skip_dodgy_days = True

In [4]:
if copOrOther:
    output = f"./output/version{version}/{copOrOther}/{filetype}/before{padbefore}_after{padafter}_norm{norm_group}"
else:
    output = f"./output/version{version}/{filetype}/before{padbefore}_after{padafter}_norm{norm_group}"
if skip_dodgy_days:
    output += "_cleaned"
Path(output).mkdir(exist_ok=True, parents=True)

In [5]:
# name of the relevant column in the company df. May be overwritten below depending on filetype. 
close = "Close"
# List of companies whose files we will read
if filetype == "renewable":
    companylist = [
        "0916.HK", "BEP", "EDPR.LS", "FSLR", "NEE", "NHPC.NS", "SUZLON.NS", "VWS.CO", "NPI.TO", "009830.KS"
    ]
elif filetype == "renewable_20":
    companylist = [
        "0916.HK", "BEP", "EDPR.LS", "FSLR", "NEE", "NHPC.NS", "SUZLON.NS", "VWS.CO", "NPI.TO", "009830.KS",
        "ORA", "3800.HK", "PLUG", "NDX1.F", "BLX.TO", "ECV.F", "SLR.MC", "S92.F", "VBK.F", "CSIQ", 
    ]
elif filetype == "fossil":
    companylist = [
        "XOM", "CVX", "SHEL", "601857.SS", "TTE", "COP", "BP", "PBR", "EQNR",  "600028.SS"
    ]
elif filetype == "fossil_20":
    companylist = [
        "XOM", "CVX", "SHEL", "601857.SS", "TTE", "COP", "BP", "PBR", "EQNR",  "600028.SS",
        "0883.HK", "SO", "ENB", "SLB", "DUK", "EOG", "CNQ", "EPD", "E", "OXY"
        
    ]
elif filetype == "constsust_20":
    companylist = ["ABT", "AMZN", "AZN", "BAC", "BRK-B", "COST", "GOOG", "JNJ", "JPM", "KO", 
                  "LLY", "MCD", "MRK", "NESN.SW", "NVO", "PEP", "PG", "ROG.SW", "TYT.L", "WMT"]
elif filetype == "control":
    companylist = ["AAPL", "AMZN", "BRK-B", "GOOG", "LLY", "MSFT", "NVDA", "TSM", "UNH", "V"]
elif filetype == "control_20":
    companylist = ["AAPL", "AMZN", "BRK-B", "GOOG", "LLY", "MSFT", "NVDA", "TSM", "UNH", "V",
                   "HD", "PG", "005930.KS", "MC.PA", "JNJ", "MA", "WMT", "AVGO", "NVO", "JPM"]
elif filetype == "oilprice":
    companylist = ["OilPrice"]
    close = "Adj Close**"
elif filetype == "tmp":
    companylist = ["^SPX"]
elif filetype == "oilfuturesApril":
    companylist = ["CrudeOilWTIFrontMonthApril"]
elif filetype == "greencred_20":
    companylist = ["005930.KS", "AAPL", "ACN", "ADBE", "AMD", "ASML", "AVGO", "CRM", "HD", "MA", 
                   "MC.PA", "MSFT", "NFLX", "NVDA", "NVS", "ORCL", "RMS.PA", "TMO", "UNH", "V"]
elif filetype == "dirty_20": 
    companylist = [
        'XOM', '600519.SS', 'CVX', 'KO', 'RELIANCE.NS',
        'SHEL', '601857.SS', 'WFC', '601288.SS', '601988.SS', 'BA', 'COP',
        'RIO', 'PBR', 'BP', '601088.SS', 'EQNR', 'MO', 'CNQ', 'ITC.NS'
    ]
else: 
    companylist = os.listdir(f"./input/{filetype}/")
    companylist = [x[:-4] for x in companylist]
    # In these cases, we expect there to be around 100 items
    if len(companylist) < 21:
        raise ValueError("Invalid filetype option")

In [6]:
if filetype[-2:] == "20":
    assert len(companylist) == 20
elif filetype == "all_companies":
    assert len(companylist) == 100

In [7]:
if norm_group:
    companynormlist = os.listdir(f"./input/{norm_group}/")
    companynormlist = [x[:-4] for x in companynormlist]
    try: 
        norm_string_end = int(norm_group.split("_")[-1])
    except:
        norm_string_end = None
    if norm_string_end:
        assert len(companynormlist) == norm_string_end

In [8]:
if not copOrOther:
    copdates = pd.read_csv("./input/CopDates.txt", delimiter="|")
    copdates = copdates.iloc[:, 1:-1]
    copdates.columns = copdates.columns.str.replace('\s+', '')
    copdates["Start"] = pd.to_datetime(copdates["Start"])
    copdates["End"] = pd.to_datetime(copdates["End"])
    meetingstring = "COP number"
    copOrOtherLongstring  = "COP"
elif copOrOther == "OPEC":
    copdates = pd.read_csv("./input/OPEC_all_2002.csv", delimiter=",", parse_dates=["Date"])
    copdates["Start"] = copdates["Date"]
    copdates["End"] = copdates["Date"]
    meetingstring = f"OPEC meeting"
    copOrOtherLongstring = meetingstring
elif copOrOther == "OPEC_Conference":
    copdates = pd.read_csv("./input/OPEC_all_2002.csv", delimiter=",", parse_dates=["Date"])
    copdates["Start"] = copdates["Date"]
    copdates["End"] = copdates["Date"]
    copdates = copdates.drop_duplicates(subset=["Date"], keep="first")
    copdates = copdates.loc[["Meeting of the OPEC Conference" in i for i in copdates["Meeting Title"]]]
    copdates = copdates.reset_index(drop=True)
    meetingstring = f"OPEC meeting"
    copOrOtherLongstring = meetingstring
elif copOrOther == "OPEC_28":
    copdates = pd.read_csv("./input/OPEC_all_2002.csv", delimiter=",", parse_dates=["Date"])
    copdates["Start"] = copdates["Date"]
    copdates["End"] = copdates["Date"]
    copdates = copdates.drop_duplicates(subset=["Date"], keep="first")
    copdates.iloc[np.arange(len(copdates) % 28, len(copdates),  (len(copdates) // 28 )), :].reset_index()
    meetingstring = f"OPEC meeting"
    copOrOtherLongstring = meetingstring
elif copOrOther == "OPEC_conf_28":
    copdates = pd.read_csv("./input/OPEC_all_2002.csv", delimiter=",", parse_dates=["Date"])
    copdates["Start"] = copdates["Date"]
    copdates["End"] = copdates["Date"]
    copdates = copdates.loc[["Meeting of the OPEC Conference" in i for i in copdates["Meeting Title"]]]
    copdates = copdates.drop_duplicates(subset=["Date"], keep="first")
    copdates.iloc[np.arange(len(copdates) % 28, len(copdates), (len(copdates) // 28 )), :].reset_index()
    meetingstring = f"OPEC meeting"
    copOrOtherLongstring = meetingstring
elif copOrOther == "OilSpill":
    copdates = pd.read_csv("./input/Oil spills data_v2.csv", delimiter=",", parse_dates=["Date"])
    copdates["Start"] = copdates["Date"]
    copdates["End"] = copdates["Date"]
    meetingstring = f"Oil spill"
    copOrOtherLongstring = meetingstring
elif copOrOther == "IPCC":
    copdates = pd.read_csv("./input/IPCC_dates.csv", delimiter=",", parse_dates=["Date"])
    copdates["Start"] = copdates["Date"]
    copdates["End"] = copdates["Date"]
    meetingstring = "IPCC report"
    copOrOtherLongstring = "IPCC report release"
else:
    raise ValueError("Did not specify a valid copOrOther")
if copOrOther:
    # In all cases, we can't allow duplicate dates so should  flush them out.
    copdates = copdates.drop_duplicates(subset=["Date"], keep="first")

In [9]:
pd.read_csv("./input/OPEC_all_2002.csv", delimiter=",", parse_dates=["Date"]).tail(40)

,Date,Meeting Title
67,2019-12-06,7th OPEC and non-OPEC Ministerial Meeting
68,2019-12-06,177th Meeting of the OPEC Conference
69,2019-12-06,7th OPEC and non-OPEC Ministerial Meeting
70,2020-03-05,178th (Extraordinary) Meeting of the OPEC Conf...
71,2020-04-09,9th (Extraordinary) OPEC and non-OPEC Minister...
72,2020-04-12,10th (Extraordinary) OPEC and non-OPEC Ministe...
73,2020-06-06,179th Meeting of the OPEC Conference
74,2020-06-06,11th OPEC and non-OPEC Ministerial Meeting
75,2020-11-30,180th Meeting of the OPEC Conference
76,2020-12-03,12th OPEC and non-OPEC Ministerial Meeting


In [10]:
if padbefore:
    copdates["Start"] = copdates["Start"] - timedelta(days=padbefore)
if padafter:
    copdates["End"] = copdates["End"] + timedelta(days=padafter)

# Read company data

In [11]:
# This reads the data from a filestring ending "filetype/company.csv" and appends it to the list results
def read_company_data(filetype, company, results):
    file_path = f'./input/{filetype}/{company}.csv'# Name of the variable denoting price at close
    # Read the data from the CSV file into a DataFrame
    df = pd.read_csv(file_path, parse_dates=['Date'], dayfirst=True)
    df = df[np.isfinite(df[close])]
    df["company"] = company
    df["DayChange"] = df[close].pct_change()
    df["DayVar"] = (df["High"]-df["Low"]) / df["High"]
    df["COP"] = np.nan
    for num, row in copdates.iterrows():
           df.loc[(row["Start"] <= df["Date"]) & (row["End"] >= df["Date"]), "COP"] = num
    results.append(df)

In [12]:
all_data = []
for company in companylist:
    read_company_data(filetype, company, all_data)
all_data = pd.concat(all_data)
all_data

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,company,DayChange,DayVar,COP
0,2000-01-04,22.765358,23.084499,21.595176,22.233458,13252265,0.0,0.0,0941.HK,NaN,0.064516,NaN
1,2000-01-05,20.382442,20.382442,18.893120,19.190983,33294893,0.0,0.0,0941.HK,-0.136842,0.073069,NaN
2,2000-01-06,19.361197,19.361197,16.595312,17.829323,36143450,0.0,0.0,0941.HK,-0.070953,0.142857,NaN
3,2000-01-07,18.339943,18.510152,17.361245,17.786766,28978733,0.0,0.0,0941.HK,-0.002387,0.062069,NaN
4,2000-01-10,19.361189,20.084574,19.148428,19.999470,30015966,0.0,0.0,0941.HK,0.124402,0.046610,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
8679,2024-06-14,66.540001,67.110001,66.300003,67.019997,12590100,0.0,0.0,WMT,0.004798,0.012070,NaN
8680,2024-06-17,66.919998,67.440002,66.410004,67.419998,12103000,0.0,0.0,WMT,0.005968,0.015273,NaN
8681,2024-06-18,67.629997,67.870003,67.300003,67.599998,12093500,0.0,0.0,WMT,0.002670,0.008398,NaN
8682,2024-06-20,67.349998,68.129997,67.300003,68.010002,13857500,0.0,0.0,WMT,0.006065,0.012183,NaN


In [13]:
assert len(all_data[all_data.Date > pd.datetime(year=2011, month=1, day=1)]["company"].unique()) == len(companylist)

C:\Users\rlamboll\AppData\Local\Temp\ipykernel_29968\2877074325.py:1: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  assert len(all_data[all_data.Date > pd.datetime(year=2011, month=1, day=1)]["company"].unique()) == len(companylist)


# Optionally clean the data

In [14]:
if any(all_data[all_data.Close < 0]):
    print("Warning: data goes negative")
    print(all_data[all_data.Close < 0])
    all_data = all_data[all_data.Close > 0]

Empty DataFrame
Columns: [Date, Open, High, Low, Close, Volume, Dividends, Stock Splits, company, DayChange, DayVar, COP]
Index: []


In [15]:
all_data[((all_data["DayChange"] <-0.5) | (all_data["DayChange"] >0.5))]

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,company,DayChange,DayVar,COP
3940,2008-10-13,11.04421,13.642426,9.061548,12.955295,199684000,0.0,0.0,MS,0.869834,0.335782,NaN


In [16]:
all_data = all_data[((all_data["DayChange"] >-0.5) & (all_data["DayChange"] < 0.5))]

In [17]:
all_data[((all_data["DayChange"] <-0.2) | (all_data["DayChange"] >0.2))]

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,company,DayChange,DayVar,COP
797,1993-02-25,2.864264,3.414652,2.853032,3.324793,177052000,0.0,0.0,AMGN,-0.200000,0.164474,NaN
34,1997-07-03,0.079948,0.095833,0.079688,0.095573,251544000,0.0,0.0,AMZN,0.203281,0.168470,NaN
79,1997-09-08,0.126563,0.151042,0.125000,0.150000,112968000,0.0,0.0,AMZN,0.200000,0.172416,NaN
326,1998-08-31,0.878646,0.896875,0.692708,0.697917,820008000,0.0,0.0,AMZN,-0.209089,0.227643,NaN
385,1998-11-23,1.590625,1.820833,1.550521,1.816667,1355628000,0.0,0.0,AMZN,0.206921,0.148455,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
1081,2011-08-02,21.724152,21.789636,16.681921,16.796518,34951000,0.0,0.0,TMUS,-0.365884,0.234410,NaN
1329,2012-07-26,11.819772,14.324516,11.787030,14.062582,16323650,0.0,0.0,TMUS,0.367834,0.177143,NaN
720,2013-05-09,4.674667,5.051333,4.246000,4.626667,429075000,0.0,0.0,TSLA,0.243951,0.159430,NaN
2566,2020-09-08,118.666664,122.913330,109.959999,110.070000,346397100,0.0,0.0,TSLA,-0.210628,0.105386,NaN


In [18]:
assert all_data.COP.sum() > len(copdates)

In [19]:
if norm_group:
    normdata = []
    for company in companynormlist:
        read_company_data(norm_group, company, normdata)
    normdata = pd.concat(normdata)
    meanNorm = normdata.groupby('Date').mean()

In [20]:
if skip_dodgy_days:
    all_data_2 = all_data[
        (all_data["Volume"]>1000)&(all_data["Volume"].shift(1)>1000)&
        (all_data["Volume"].shift(-1)>1000)&(all_data["Stock Splits"]==0)
    ]
    deleted_data = all_data[
        ~((all_data["Volume"]>1000)&(all_data["Volume"].shift(1)>1000)&
        (all_data["Volume"].shift(-1)>1000)&(all_data["Stock Splits"]==0))
    ]
    all_data = all_data_2
    if norm_group:
        normdata[
            (normdata["Volume"]>1000)&(normdata["Volume"].shift(1)>1000)&
            (normdata["Volume"].shift(-1)>1000)&(normdata["Stock Splits"]==0)
        ]
        meanNorm = normdata.groupby('Date').mean()

In [21]:
# If we have data about ethics ratings we can also add this
if filetype == "all_companies":
    company_ethics = pd.read_csv("./input/companiesmarketcap.com - Companies ranked by Market Cap - CompaniesMarketCap.com.csv")
    company_ethics = company_ethics.loc[:, ["Symbol", "Sustainalytics value"]]
    all_data = pd.merge(all_data, company_ethics, left_on="company", right_on="Symbol")
    del all_data["Symbol"]
    company_ethics["Sustainalytics value"].hist()

In [22]:
all_data

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,company,DayChange,DayVar,COP
2,2000-01-06,19.361197,19.361197,16.595312,17.829323,36143450,0.0,0.0,0941.HK,-0.070953,0.142857,NaN
3,2000-01-07,18.339943,18.510152,17.361245,17.786766,28978733,0.0,0.0,0941.HK,-0.002387,0.062069,NaN
4,2000-01-10,19.361189,20.084574,19.148428,19.999470,30015966,0.0,0.0,0941.HK,0.124402,0.046610,NaN
5,2000-01-11,20.595202,21.105825,20.212232,20.552649,17250500,0.0,0.0,0941.HK,0.027660,0.042339,NaN
6,2000-01-12,19.956920,20.084576,19.659055,20.042023,13980916,0.0,0.0,0941.HK,-0.024845,0.021186,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
8678,2024-06-13,66.300003,66.760002,65.949997,66.699997,11196600,0.0,0.0,WMT,0.005881,0.012133,NaN
8679,2024-06-14,66.540001,67.110001,66.300003,67.019997,12590100,0.0,0.0,WMT,0.004798,0.012070,NaN
8680,2024-06-17,66.919998,67.440002,66.410004,67.419998,12103000,0.0,0.0,WMT,0.005968,0.015273,NaN
8681,2024-06-18,67.629997,67.870003,67.300003,67.599998,12093500,0.0,0.0,WMT,0.002670,0.008398,NaN


In [23]:
all_cat = all_data.copy()
if year_company_data:
    all_cat["Year_company"] = all_cat.Date.dt.year.astype(str) + "_" + all_cat["company"]
    del all_cat["company"]
else:
    all_cat["Year"] = all_cat.Date.dt.year.astype(str)

In [24]:
all_cat

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,DayChange,DayVar,COP,Year_company
2,2000-01-06,19.361197,19.361197,16.595312,17.829323,36143450,0.0,0.0,-0.070953,0.142857,NaN,2000_0941.HK
3,2000-01-07,18.339943,18.510152,17.361245,17.786766,28978733,0.0,0.0,-0.002387,0.062069,NaN,2000_0941.HK
4,2000-01-10,19.361189,20.084574,19.148428,19.999470,30015966,0.0,0.0,0.124402,0.046610,NaN,2000_0941.HK
5,2000-01-11,20.595202,21.105825,20.212232,20.552649,17250500,0.0,0.0,0.027660,0.042339,NaN,2000_0941.HK
6,2000-01-12,19.956920,20.084576,19.659055,20.042023,13980916,0.0,0.0,-0.024845,0.021186,NaN,2000_0941.HK
...,...,...,...,...,...,...,...,...,...,...,...,...
8678,2024-06-13,66.300003,66.760002,65.949997,66.699997,11196600,0.0,0.0,0.005881,0.012133,NaN,2024_WMT
8679,2024-06-14,66.540001,67.110001,66.300003,67.019997,12590100,0.0,0.0,0.004798,0.012070,NaN,2024_WMT
8680,2024-06-17,66.919998,67.440002,66.410004,67.419998,12103000,0.0,0.0,0.005968,0.015273,NaN,2024_WMT
8681,2024-06-18,67.629997,67.870003,67.300003,67.599998,12093500,0.0,0.0,0.002670,0.008398,NaN,2024_WMT


In [25]:
all_cat = pd.get_dummies(all_cat, drop_first=True)
all_cat["COP"] = [1 if x == x else 0 for x in all_cat["COP"]]
all_cat

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,DayChange,DayVar,...,Year_company_2024_PEP,Year_company_2024_PG,Year_company_2024_PM,Year_company_2024_ROG.SW,Year_company_2024_TM,Year_company_2024_TMUS,Year_company_2024_TSLA,Year_company_2024_TTE,Year_company_2024_TXN,Year_company_2024_WMT
2,2000-01-06,19.361197,19.361197,16.595312,17.829323,36143450,0.0,0.0,-0.070953,0.142857,...,0,0,0,0,0,0,0,0,0,0
3,2000-01-07,18.339943,18.510152,17.361245,17.786766,28978733,0.0,0.0,-0.002387,0.062069,...,0,0,0,0,0,0,0,0,0,0
4,2000-01-10,19.361189,20.084574,19.148428,19.999470,30015966,0.0,0.0,0.124402,0.046610,...,0,0,0,0,0,0,0,0,0,0
5,2000-01-11,20.595202,21.105825,20.212232,20.552649,17250500,0.0,0.0,0.027660,0.042339,...,0,0,0,0,0,0,0,0,0,0
6,2000-01-12,19.956920,20.084576,19.659055,20.042023,13980916,0.0,0.0,-0.024845,0.021186,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8678,2024-06-13,66.300003,66.760002,65.949997,66.699997,11196600,0.0,0.0,0.005881,0.012133,...,0,0,0,0,0,0,0,0,0,1
8679,2024-06-14,66.540001,67.110001,66.300003,67.019997,12590100,0.0,0.0,0.004798,0.012070,...,0,0,0,0,0,0,0,0,0,1
8680,2024-06-17,66.919998,67.440002,66.410004,67.419998,12103000,0.0,0.0,0.005968,0.015273,...,0,0,0,0,0,0,0,0,0,1
8681,2024-06-18,67.629997,67.870003,67.300003,67.599998,12093500,0.0,0.0,0.002670,0.008398,...,0,0,0,0,0,0,0,0,0,1


In [26]:
all_cat = all_cat[all_cat.Date > pd.datetime(year=1995, month=1, day=1)]
if norm_group:
    all_cat = pd.merge(all_cat, meanNorm.loc[:, ["DayChange", "DayVar"]].reset_index(), on="Date")
all_cat = all_cat[np.isnan(all_cat).sum(axis=1)==0]

C:\Users\rlamboll\AppData\Local\Temp\ipykernel_29968\4055006351.py:1: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  all_cat = all_cat[all_cat.Date > pd.datetime(year=1995, month=1, day=1)]


In [27]:
all_cat = pd.get_dummies(all_cat)

In [28]:
# If we have data about ethics ratings we can also add an interaction between this and COPs
if filetype == "all_companies":
    all_cat["COP_good_ethics"] = all_cat["COP"] * (all_cat["Sustainalytics value"] < 20)
    all_cat["COP_mid_ethics"] = all_cat["COP"] * (all_cat["Sustainalytics value"] > 20) * (all_cat["Sustainalytics value"] < 30)
    all_cat["COP_bad_ethics"] = all_cat["COP"] * (all_cat["Sustainalytics value"] > 30)
    # Since all companies are covered by one of these, we remove the general case to prevent colinearity issues
    del all_cat["COP"]

In [29]:
if not norm_group:
        target = "DayChange"
        ignore_col = "DayVar"
        X_train = all_cat.loc[:, all_cat.columns != ignore_col].iloc[:, 9:]
else:
    target = "DayChange_x"
    ignore_col = "DayVar_y"
    X_train = all_cat.loc[:, all_cat.columns != ignore_col].iloc[:, 10:]
    # We have established that this works so no longer use the test-train distinction
    y_train = all_cat[target]
if do_Test_train_split:
    X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [30]:
ls=sm.OLS(y_train, X_train).fit()

In [31]:
ls.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            DayChange_x   R-squared:                       0.284
Model:                            OLS   Adj. R-squared:                  0.281
Method:                 Least Squares   F-statistic:                     96.53
Date:                Fri, 13 Sep 2024   Prob (F-statistic):               0.00
Time:                        15:26:27   Log-Likelihood:             6.0465e+05
No. Observations:              227141   AIC:                        -1.207e+06
Df Residuals:                  226209   BIC:                        -1.198e+06
Df Model:                         931                                         
Covariance Type:            nonrobust                                         
===============================================================================================
                                  coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
COP                          -2.37e-06      0.000     -0.008      0.994      -0.001       0.001
Year_company_1990_AMGN       -2.61e-17   3.79e-18     -6.894      0.000   -3.35e-17   -1.87e-17
Year_company_1990_BAC        4.964e-16   1.03e-17     48.375      0.000    4.76e-16    5.17e-16
Year_company_1990_BHP       -6.636e-16   3.27e-18   -202.947      0.000    -6.7e-16   -6.57e-16
Year_company_1990_CAT       -1.003e-15    3.7e-18   -270.826      0.000   -1.01e-15   -9.95e-16
Year_company_1990_CMCSA     -3.344e-16   2.21e-18   -151.108      0.000   -3.39e-16    -3.3e-16
Year_company_1990_COST       5.867e-16    3.9e-18    150.300      0.000    5.79e-16    5.94e-16
Year_company_1990_JNJ        2.499e-16   2.19e-18    114.258      0.000    2.46e-16    2.54e-16
Year_company_1990_JPM         3.25e-16   1.29e-17     25.173      0.000       3e-16     3.5e-16
Year_company_1990_LLY        6.949e-18   9.99e-18      0.696      0.487   -1.26e-17    2.65e-17
Year_company_1990_MCD       -7.682e-18   1.28e-17     -0.598      0.550   -3.28e-17    1.75e-17
Year_company_1990_MRK        -7.38e-18   1.54e-17     -0.480      0.631   -3.75e-17    2.28e-17
Year_company_1990_NVO       -2.465e-18   1.07e-17     -0.230      0.818   -2.35e-17    1.86e-17
Year_company_1990_PEP        2.416e-17    1.2e-16      0.202      0.840    -2.1e-16    2.59e-16
Year_company_1990_PG        -3.967e-19   2.43e-18     -0.163      0.870   -5.16e-18    4.37e-18
Year_company_1990_TM        -2.732e-18   4.06e-18     -0.674      0.500   -1.07e-17    5.22e-18
Year_company_1990_TXN        2.695e-18   2.25e-18      1.199      0.230   -1.71e-18     7.1e-18
Year_company_1990_WMT         6.76e-18   2.08e-18      3.246      0.001    2.68e-18    1.08e-17
Year_company_1991_ABT       -8.778e-18   2.34e-18     -3.746      0.000   -1.34e-17   -4.19e-18
Year_company_1991_AMGN       3.666e-19   1.61e-18      0.228      0.820   -2.79e-18    3.53e-18
Year_company_1991_BAC        6.336e-19    1.5e-18      0.423      0.672    -2.3e-18    3.57e-18
Year_company_1991_BHP        6.914e-19    1.5e-18      0.461      0.645   -2.25e-18    3.63e-18
Year_company_1991_CAT       -5.934e-19   1.62e-18     -0.366      0.715   -3.77e-18    2.59e-18
Year_company_1991_CMCSA     -1.536e-18   1.57e-18     -0.981      0.327    -4.6e-18    1.53e-18
Year_company_1991_COST      -9.729e-19   1.44e-18     -0.674      0.500    -3.8e-18    1.85e-18
Year_company_1991_JNJ        3.278e-18   1.59e-18      2.065      0.039    1.67e-19    6.39e-18
Year_company_1991_JPM       -3.156e-19   1.75e-18     -0.180      0.857   -3.75e-18    3.12e-18
Year_company_1991_LLY        1.949e-18   1.48e-18      1.320      0.187   -9.45e-19    4.84e-18
Year_company_1991_MCD        8.644e-19    1.2e-18      0.719      0.472   -1.49e-18    3.22e-18
Year_compan

In [32]:
if do_Test_train_split:
    model = LinearRegression()
    model.fit(X_train,y_train)

In [33]:
summary_str = ls.summary().as_text()
with open(output + "/OLSsummary.txt", "w") as f:
    f.write(summary_str)

In [34]:
# Now do the same for daily variability
if not norm_group:
    target = "DayVar"
    ignore_col = "DayChange"
    accept_index_dayvar = (all_cat[target] >= 0) & (all_cat[target] < 0.5)
    X2_train = all_cat.loc[accept_index_dayvar, all_cat.columns != ignore_col].iloc[:, 9:]
else:
    target = "DayVar_x"
    ignore_col = "DayChange_y"
    accept_index_dayvar = (all_cat[target] >= 0) & (all_cat[target] < 0.5)
    X2_train = all_cat.loc[accept_index_dayvar, all_cat.columns != ignore_col].iloc[:, 10:]
y2_train = all_cat.loc[accept_index_dayvar, target]

In [35]:
ls2=sm.OLS(y2_train, X2_train).fit()

In [36]:
ls2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               DayVar_x   R-squared:                       0.617
Model:                            OLS   Adj. R-squared:                  0.615
Method:                 Least Squares   F-statistic:                     391.4
Date:                Fri, 13 Sep 2024   Prob (F-statistic):               0.00
Time:                        15:28:37   Log-Likelihood:             7.2762e+05
No. Observations:              227138   AIC:                        -1.453e+06
Df Residuals:                  226206   BIC:                        -1.444e+06
Df Model:                         931                                         
Covariance Type:            nonrobust                                         
===============================================================================================
                                  coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
COP                         -1.796e-05      0.000     -0.098      0.922      -0.000       0.000
Year_company_1990_AMGN       -2.68e-16   1.85e-18   -144.625      0.000   -2.72e-16   -2.64e-16
Year_company_1990_BAC        4.643e-16   1.86e-18    249.041      0.000    4.61e-16    4.68e-16
Year_company_1990_BHP        5.679e-16   4.36e-18    130.102      0.000    5.59e-16    5.76e-16
Year_company_1990_CAT       -5.496e-16   3.54e-18   -155.118      0.000   -5.56e-16   -5.43e-16
Year_company_1990_CMCSA     -5.135e-16   3.76e-18   -136.494      0.000   -5.21e-16   -5.06e-16
Year_company_1990_COST       4.107e-16   3.31e-18    124.183      0.000    4.04e-16    4.17e-16
Year_company_1990_JNJ        6.804e-17   1.06e-18     64.220      0.000     6.6e-17    7.01e-17
Year_company_1990_JPM         1.25e-16   2.48e-18     50.461      0.000     1.2e-16     1.3e-16
Year_company_1990_LLY        1.474e-16   7.63e-18     19.320      0.000    1.32e-16    1.62e-16
Year_company_1990_MCD       -3.071e-16   3.43e-18    -89.508      0.000   -3.14e-16      -3e-16
Year_company_1990_MRK       -1.872e-16   1.55e-17    -12.051      0.000   -2.18e-16   -1.57e-16
Year_company_1990_NVO        7.106e-18   3.22e-18      2.210      0.027    8.03e-19    1.34e-17
Year_company_1990_PEP       -1.272e-16   2.01e-17     -6.319      0.000   -1.67e-16   -8.77e-17
Year_company_1990_PG        -6.028e-17   8.65e-18     -6.965      0.000   -7.72e-17   -4.33e-17
Year_company_1990_TM        -3.352e-17    3.1e-18    -10.819      0.000   -3.96e-17   -2.74e-17
Year_company_1990_TXN        6.862e-20   9.84e-19      0.070      0.944   -1.86e-18       2e-18
Year_company_1990_WMT        8.304e-18   1.05e-18      7.931      0.000    6.25e-18    1.04e-17
Year_company_1991_ABT        7.794e-18   1.56e-18      4.987      0.000    4.73e-18    1.09e-17
Year_company_1991_AMGN      -1.782e-17   1.05e-18    -16.921      0.000   -1.99e-17   -1.58e-17
Year_company_1991_BAC       -4.781e-18   8.87e-19     -5.388      0.000   -6.52e-18   -3.04e-18
Year_company_1991_BHP         -4.5e-18   8.12e-19     -5.544      0.000   -6.09e-18   -2.91e-18
Year_company_1991_CAT         1.58e-18   8.39e-19      1.883      0.060   -6.46e-20    3.22e-18
Year_company_1991_CMCSA     -8.162e-18   1.13e-18     -7.233      0.000   -1.04e-17   -5.95e-18
Year_company_1991_COST      -8.431e-18   8.92e-19     -9.457      0.000   -1.02e-17   -6.68e-18
Year_company_1991_JNJ        7.432e-18    1.1e-18      6.746      0.000    5.27e-18    9.59e-18
Year_company_1991_JPM        1.993e-17   1.27e-18     15.739      0.000    1.74e-17    2.24e-17
Year_company_1991_LLY       -1.038e-17   8.35e-19    -12.420      0.000    -1.2e-17   -8.74e-18
Year_company_1991_MCD        3.729e-18   8.32e-19      4.481      0.000     2.1e-18    5.36e-18
Year_compan

In [37]:
summary_str2 = ls2.summary().as_text()
with open(output + "/OLSsummaryDailyVar.txt", "w") as f:
    f.write(summary_str2)

In [38]:
if do_Test_train_split:
    # Displaying the test data
    y_pred = model.predict(X_test)
    test_data = pd.DataFrame({
        'y_test': y_test,
        'y_pred': y_pred
    })

    m, b = np.polyfit(y_test, y_pred, 1)  # m is slope, b is intercept
    plt.scatter(y_test, y_pred, color='blue')
    X = np.array([y_test.min(), y_test.max()])
    plt.plot(X, m*X + b, color='red', label='Line of Best Fit')
    plt.plot(X, X, linestyle="--")
    print(f"gradient is {m}")

In [39]:
output

'./output/version16/OPEC/constsust_34/before0_after0_normconstsust_34_cleaned'

In [40]:
copdates

,Date,Meeting Title,Start,End
0,2002-03-15,119th Meeting of the OPEC Conference,2002-03-15,2002-03-15
1,2002-06-26,120th (Extraordinary) Meeting of the OPEC Conf...,2002-06-26,2002-06-26
2,2002-09-19,121st Meeting of the OPEC Conference,2002-09-19,2002-09-19
3,2002-12-12,122nd (Extraordinary) Meeting of the OPEC Conf...,2002-12-12,2002-12-12
4,2003-01-12,123rd (Extraordinary) Meeting of the OPEC Conf...,2003-01-12,2003-01-12
...,...,...,...,...
102,2023-10-04,50th JMMC Meeting,2023-10-04,2023-10-04
103,2023-10-09,World Oil Outlook 2023 Launch,2023-10-09,2023-10-09
104,2023-11-09,6th High-level Meeting of the OPEC-India Energ...,2023-11-09,2023-11-09
105,2023-11-30,36th OPEC and non-OPEC Ministerial Meeting,2023-11-30,2023-11-30
